In [1]:
import pandas as pd
import json
import duckdb

In [ ]:
! pip install langgraph langchain-community pypdf faiss-cpu sentence-transformers langchain-groq

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [17]:
# define the path to the transactional data
TRANSACTION_DATA_FILE_PATH = '/content/Titanic-Dataset.csv'

In [6]:
# define the path to the transactional data
TRANSACTION_DATA_FILE_PATH = '/content/Titanic-Dataset.csv'

In [8]:
from groq import Groq

client = Groq()

In [13]:
llm="deepseek-r1-distill-llama-70b"

In [57]:
import pandas as pd
import duckdb
from groq import Groq
# from helper import get_groq_api_key  # Ensure this function fetches the Groq API key

# Initialize the Groq client
groq_api_key = "gsk_TGJDpgzTUYmkCsEMcJeXWGdyi"
client = Groq(api_key=groq_api_key)

# Define the model name
MODEL = "gemma2-9b-it"

# Define the path to the Titanic dataset
TRANSACTION_DATA_FILE_PATH = "/content/Titanic-Dataset.csv"

# SQL generation prompt template
SQL_GENERATION_PROMPT = """
Generate an SQL query based on a prompt. Do not reply with anything besides the SQL query.
The prompt is: {prompt}

The available columns are: {columns}
The table name is: {table_name}
"""

# Function to generate SQL query using Groq's streaming format
def generate_sql_query(prompt: str, columns: list, table_name: str) -> str:
    """Generate an SQL query using Groq's streaming LLM"""
    formatted_prompt = SQL_GENERATION_PROMPT.format(
        prompt=prompt, columns=columns, table_name=table_name
    )

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": formatted_prompt}],
        temperature=0.6,
        max_completion_tokens=512,
        top_p=0.95,
        stream=True,  # Enable streaming
    )

    sql_query = ""
    for chunk in response:
        sql_query += chunk.choices[0].delta.content or ""  # Extract the response incrementally

    # Clean the SQL response
    return sql_query.strip().replace("```sql", "").replace("```", "")

# Function to query Titanic data using SQL and DuckDB
def lookup_data(prompt: str) -> str:
    """Query Titanic dataset using SQL generated by Groq"""
    try:
        table_name = "titanic1"

        # Load the dataset
        df = pd.read_csv(TRANSACTION_DATA_FILE_PATH)

        # Create a DuckDB in-memory connection
        con = duckdb.connect()

        # Register the DataFrame as a table (correct way)
        con.register(table_name, df)

        # Generate SQL query
        sql_query = generate_sql_query(prompt, df.columns.tolist(), table_name)

        # Execute SQL query
        result = con.execute(sql_query).fetchdf()

        return result.to_string()

    except Exception as e:
        return f"Error accessing data: {str(e)}"

# Example usage
example_data = lookup_data("List out the column names in Titanic data")
print(example_data)


    column_name
0   PassengerId
1      Survived
2        Pclass
3          Name
4           Sex
5           Age
6         SibSp
7         Parch
8        Ticket
9          Fare
10        Cabin
11     Embarked


In [58]:
example_data = lookup_data("How many male and female are  in the titanic dataset?")
print(example_data)

   male_count  female_count
0         577           314


In [59]:
example_data = lookup_data("Find the missing values count  in the titanic dataset?")
print(example_data)

   Missing_Age  Missing_Cabin
0          177            687


In [60]:
example_data = lookup_data("Give the summary about the dataset and find the insights from the dataset")
print(example_data)

   average_survival_rate  total_passengers  number_survivors  number_deceased  total_fare  max_age  min_age  number_of_passenger_classes
0               0.383838               891               342              549  28693.9493     80.0     0.42                            3
